In [1]:
from google.colab import files

uploaded = files.upload()

Saving Zomato Dataset.csv to Zomato Dataset.csv


In [2]:
import pandas as pd

# Read the uploaded CSV
df_raw = pd.read_csv("Zomato Dataset.csv")

# Check that it loaded correctly
print("Dataset loaded successfully!")
print("Shape:", df_raw.shape)
df_raw.head()

Dataset loaded successfully!
Shape: (45584, 20)


,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Order_Date,Time_Orderd,Time_Order_picked,Weather_conditions,Road_traffic_density,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,City,Time_taken (min)
0,0xcdcd,DEHRES17DEL01,36.0,4.2,30.327968,78.046106,30.397968,78.116106,12-02-2022,21:55,22:10,Fog,Jam,2,Snack,motorcycle,3.0,No,Metropolitian,46
1,0xd987,KOCRES16DEL01,21.0,4.7,10.003064,76.307589,10.043064,76.347589,13-02-2022,14:55,15:05,Stormy,High,1,Meal,motorcycle,1.0,No,Metropolitian,23
2,0x2784,PUNERES13DEL03,23.0,4.7,18.562450,73.916619,18.652450,74.006619,04-03-2022,17:30,17:40,Sandstorms,Medium,1,Drinks,scooter,1.0,No,Metropolitian,21
3,0xc8b6,LUDHRES15DEL02,34.0,4.3,30.899584,75.809346,30.919584,75.829346,13-02-2022,09:20,09:30,Sandstorms,Low,0,Buffet,motorcycle,0.0,No,Metropolitian,20
4,0xdb64,KNPRES14DEL02,24.0,4.7,26.463504,80.372929,26.593504,80.502929,14-02-2022,19:50,20:05,Fog,Jam,1,Snack,scooter,1.0,No,Metropolitian,41


In [3]:
def generate_metadata_report(df, name):
    print(f"\n--- Metadata Report: {name} ---")
    profile = pd.DataFrame({
        "Data_Type": df.dtypes,
        "Non_Null_Count": df.count(),
        "Cardinality (Unique)": df.nunique(),
        "Sample_Value": df.iloc[0]
    })
    print(profile)
    return profile

# Generate pre-ETL report
raw_metadata = generate_metadata_report(df_raw, "Zomato Raw Data")


--- Metadata Report: Zomato Raw Data ---
                            Data_Type  Non_Null_Count  Cardinality (Unique)  \
ID                             object           45584                 45584   
Delivery_person_ID             object           45584                  1320   
Delivery_person_Age           float64           43730                    22   
Delivery_person_Ratings       float64           43676                    28   
Restaurant_latitude           float64           45584                   657   
Restaurant_longitude          float64           45584                   518   
Delivery_location_latitude    float64           45584                  4373   
Delivery_location_longitude   float64           45584                  4373   
Order_Date                     object           45584                    44   
Time_Orderd                    object           43853                   176   
Time_Order_picked              object           45584                   193   
Weather_co

In [4]:
# TRANSFORM – Data Cleaning

clean_df = df_raw.copy()

In [5]:
# 2.1 Handle multiple_deliveries
clean_df["multiple_deliveries"] = pd.to_numeric(
    clean_df["multiple_deliveries"], errors="coerce"
)
clean_df["multiple_deliveries"].fillna(0, inplace=True)
clean_df["multiple_deliveries"] = clean_df["multiple_deliveries"].astype(int)

/tmp/ipython-input-3639756835.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  clean_df["multiple_deliveries"].fillna(0, inplace=True)


In [6]:
# 2.2 Handle Other Missing Numeric Values
numeric_columns = ["Delivery_person_Age", "Delivery_person_Ratings"]
for col in numeric_columns:
    clean_df[col].fillna(clean_df[col].median(), inplace=True)

/tmp/ipython-input-2433335146.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  clean_df[col].fillna(clean_df[col].median(), inplace=True)


In [7]:
# 2.3 Remove Invalid Ages
clean_df = clean_df[
    (clean_df["Delivery_person_Age"] >= 18) &
    (clean_df["Delivery_person_Age"] <= 65) ]

In [10]:

import numpy as np



In [11]:
# 2.4 Handle Outliers (Delivery Time)
cap_value = clean_df["Time_taken (min)"].quantile(0.99)
clean_df["Time_taken (min)"] = np.where(
    clean_df["Time_taken (min)"] > cap_value,
    cap_value,
    clean_df["Time_taken (min)"]
)

In [14]:
def convert_to_time(val):
    """
    Convert any Time_Orderd value to datetime.time in HH:MM format
    Handles:
    - Decimal floats: 0.375 → 09:00
    - Decimal strings: "0.375" → 09:00
    - Time strings: "08:15" → 08:15
    - Missing: "naan", NaN → NaT
    """
    # Handle missing values
    if pd.isna(val):
        return pd.NaT

    # Convert to string for consistent processing
    if not isinstance(val, str):
        val = str(val)

    val = val.strip()

    # Handle missing indicators
    if val.lower() in ['naan', 'nan', 'null', '', 'none', 'na']:
        return pd.NaT

    # CASE 1: Already in HH:MM format
    if ':' in val:
        try:
            # Parse HH:MM format
            return pd.to_datetime(val, format="%H:%M").time()
        except:
            # Try more flexible parsing if needed
            try:
                return pd.to_datetime(val).time()
            except:
                return pd.NaT

    # CASE 2: Decimal format (0.375)
    try:
        # Convert to float
        num = float(val)

        # Excel time conversion: fraction of day to hours/minutes
        total_hours = num * 24
        hours = int(total_hours)
        minutes = int((total_hours - hours) * 60)

        # Handle 24:00 case (1.0 = midnight)
        if hours == 24:
            hours = 0

        # Create time object
        return pd.Timestamp(hour=hours, minute=minutes).time()
    except:
        return pd.NaT

# Apply to both time columns
clean_df["Time_Orderd"] = clean_df["Time_Orderd"].apply(convert_to_time)
clean_df["Time_Order_picked"] = clean_df["Time_Order_picked"].apply(convert_to_time)

# Check results
print("CONVERSION RESULTS:")
print("-" * 50)
print(f"Time_Orderd - Missing: {clean_df['Time_Orderd'].isna().sum()}/{len(clean_df)}")
print(f"Time_Order_picked - Missing: {clean_df['Time_Order_picked'].isna().sum()}/{len(clean_df)}")

# Show sample conversions
print("\nSAMPLE CONVERSIONS (Time_Orderd):")
samples = clean_df["Time_Orderd"].dropna().head(10)
for i, time_val in enumerate(samples, 1):
    print(f"{i:2}. {time_val}")


CONVERSION RESULTS:
--------------------------------------------------
Time_Orderd - Missing: 5761/45546
Time_Order_picked - Missing: 5004/45546

SAMPLE CONVERSIONS (Time_Orderd):
 1. 21:55:00
 2. 14:55:00
 3. 17:30:00
 4. 09:20:00
 5. 19:50:00
 6. 20:25:00
 7. 14:55:00
 8. 20:30:00
 9. 20:40:00
10. 21:15:00


In [20]:

# Fill missing values for categorical columns

# Columns to clean
categorical_cols = ['Road_traffic_density', 'Weather_conditions', 'City', 'Festival']

# Step 1: Fill actual NaN values with 'Not Specified'
for col in categorical_cols:
    clean_df[col] = clean_df[col].fillna('Not Specified')

# Step 2: Handle other null-like strings
null_values = ['', 'nan', 'NaN', 'null', 'Null', 'NA', 'na', 'naan', 'none', 'None']

for col in categorical_cols:
    clean_df[col] = clean_df[col].replace(null_values, 'Not Specified')

    # Step 3: Optional – standardize text (capitalize first letter of each word)
    clean_df[col] = clean_df[col].str.title()


In [21]:
# Final Check
print("Cleaned dataset shape:", clean_df.shape)
print(clean_df.describe())

Cleaned dataset shape: (45546, 20)
       Delivery_person_Age  Delivery_person_Ratings  Restaurant_latitude  \
count         45546.000000             45546.000000         45546.000000   
mean             29.596693                 4.639580            17.027378   
std               5.682990                 0.310783             8.170664   
min              20.000000                 2.500000           -30.905562   
25%              25.000000                 4.600000            12.933284   
50%              30.000000                 4.700000            18.551440   
75%              34.000000                 4.800000            22.728163   
max              50.000000                 6.000000            30.914057   

       Restaurant_longitude  Delivery_location_latitude  \
count          45546.000000                45546.000000   
mean              70.253071                   17.467463   
std               22.813150                    7.336006   
min              -88.366217                 

In [22]:
#Load – Save Cleaned Dataset
clean_df.to_csv("zomato_cleaned.csv", index=False)
print("Cleaned dataset saved successfully.")
print("Cleaned dataset shape:", clean_df.shape)

Cleaned dataset saved successfully.
Cleaned dataset shape: (45546, 20)


In [23]:
#Download
from google.colab import files
files.download("zomato_cleaned.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [24]:
#Post-ETL Metadata Report
clean_metadata = generate_metadata_report(clean_df, "Zomato Cleaned Data")



--- Metadata Report: Zomato Cleaned Data ---
                            Data_Type  Non_Null_Count  Cardinality (Unique)  \
ID                             object           45546                 45546   
Delivery_person_ID             object           45546                  1320   
Delivery_person_Age           float64           45546                    21   
Delivery_person_Ratings       float64           45546                    27   
Restaurant_latitude           float64           45546                   654   
Restaurant_longitude          float64           45546                   514   
Delivery_location_latitude    float64           45546                  4373   
Delivery_location_longitude   float64           45546                  4373   
Order_Date                     object           45546                    44   
Time_Orderd                    object           39785                   160   
Time_Order_picked              object           40542                   174   
Weathe